<a href="https://colab.research.google.com/github/Giovannacm/nature-inspired-computing/blob/main/ACO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Universidade Estadual Paulista - UNESP

Programa de Pós-Graduação em Ciência da Computação

Disciplina: **Computação Inspirada pela Natureza** - Professor Fabricio Breve

**Trabalho 3** - Giovanna Carreira Marinho

# Funções auxiliares

In [1]:
import random
import numpy as np

In [2]:
def getFeromonio(grafo, aresta_i, aresta_j):
  return grafo[aresta_i][aresta_j]

In [3]:
def accFeromonio(grafo, aresta_i, aresta_j, valor):
  if grafo[aresta_i][aresta_j] > 0:
    grafo[aresta_i][aresta_j] += valor
    grafo[aresta_j][aresta_i] += valor

In [4]:
def evaporaFeromonios(grafo, valor):
  len_vertices = len(grafo)
  for i in range(len_vertices):
    for j in range(len_vertices):
      if grafo[i][j] > 0:
        grafo[i][j] *= valor

In [5]:
def getVerticesAdjacentes(grafo, aresta):
  vizinhos = []
  feromonio = []
  len_vertices = len(grafo[aresta])
  for i in range(len_vertices):
    if grafo[aresta][i] > 0:
      vizinhos.append(i)
      feromonio.append(grafo[aresta][i])
  return vizinhos, feromonio

In [6]:
def geraPopulacao(tamanho, valor):
  formigas = []
  for i in range(tamanho):
    formigas.append([valor])
  return formigas

In [21]:
def buscaMenorCaminho(grafo, p, origem, destino, quant_pop, max_it):
  t = 0
  while t < max_it:

    formigas = geraPopulacao(quant_pop, origem)

    for formiga in formigas:
      historico = formiga.copy()

      while destino not in formiga:
        vizinhos_real, feromonios_real = getVerticesAdjacentes(grafo, formiga[-1])

        vizinhos, feromonios = [], []
        for vertice in range(len(vizinhos_real)):
          if vizinhos_real[vertice] not in historico:
            vizinhos.append(vizinhos_real[vertice])
            feromonios.append(feromonios_real[vertice])

        proba = []
        for vertice in range(len(vizinhos)):
          proba.append(getFeromonio(grafo, formiga[-1], vizinhos[vertice]) / sum(feromonios))

        escolha = vizinhos[np.random.choice(np.arange(len(vizinhos)), p = proba)]

        formiga.append(escolha)
        historico.append(escolha)

    evaporaFeromonios(grafo, p)

    for formiga in formigas:
      for vertice in range(len(formiga) - 1):
        accFeromonio(grafo, formiga[vertice], formiga[vertice+1], 1/(len(formiga) - 1))

    print('\n--> Iteração:', t)
    for linha in grafo:
      print(linha)
    
    t += 1

# Atividade da aula

In [24]:
grafo = [[0, 1, 1, 1],
         [1, 0, 1, 1],
         [1, 1, 0, 1],
         [1, 1, 1, 0]]
buscaMenorCaminho(grafo=grafo, p=0.5, origem=0, destino=3, quant_pop=5, max_it=2)


--> Iteração: 0
[0, 1.0, 1.1666666666666665, 2.5]
[1.0, 0, 1.1666666666666665, 1.6666666666666665]
[1.1666666666666665, 1.1666666666666665, 0, 0.5]
[2.5, 1.6666666666666665, 0.5, 0]

--> Iteração: 1
[0, 1.0, 0.9166666666666665, 4.25]
[1.0, 0, 0.9166666666666665, 1.6666666666666665]
[0.9166666666666665, 0.9166666666666665, 0, 0.25]
[4.25, 1.6666666666666665, 0.25, 0]
